# First assignment

#### Packages and libraries installations

In [37]:
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         154 KB  anaconda

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0  conda-forge --> 2019.9.11-py36_0 anaconda
    openssl: 1.1.1c-h516909a_0 conda-forge --> 1.1.1-h7b6447c_0 anaconda


certifi-2019.9.11    | 154 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [38]:
import requests
import pandas as pd
from bs4 import BeautifulSoup



#### Importing, reading and checking data

In [39]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

In [40]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [41]:
df.shape

(288, 3)

### Grouping by Postcode first, aggregating neighbourhoods and adding the borough later

In [42]:
tt = df.groupby('Postcode', as_index=False).agg({'Neighbourhood':', '.join})
tt.head()

,Postcode,Neighbourhood
0,M1A,Not assigned
1,M1B,"Rouge, Malvern"
2,M1C,"Highland Creek, Rouge Hill, Port Union"
3,M1E,"Guildwood, Morningside, West Hill"
4,M1G,Woburn
5,M1H,Cedarbrae
6,M1J,Scarborough Village
7,M1K,"East Birchmount Park, Ionview, Kennedy Park"
8,M1L,"Clairlea, Golden Mile, Oakridge"
9,M1M,"Cliffcrest, Cliffside, Scarborough Village West"


In [43]:
tt['Borough'] = tt.Postcode.map(df.set_index('Postcode')['Borough'].to_dict())

In [44]:
tt.shape

(180, 3)

#### Rearranging the column indexes

In [45]:
tt = tt[['Postcode','Borough','Neighbourhood']]

#### This will drop all rows with not assigned Boroughs and Neighbourhoods

In [46]:
borind = tt[(tt['Borough'] == 'Not assigned')&(tt['Neighbourhood'] == 'Not assigned')].index

In [47]:
tt.drop(borind, inplace=True)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [48]:
tt.head()

,Postcode,Borough,Neighbourhood
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


#### Check for Not assigned Borough and Neighbourhoods

In [49]:
print(tt[(tt['Borough'] == 'Not assigned')].index.values)
print(tt[(tt['Neighbourhood'] == 'Not assigned')].index.values)

[]
[120]


One assigned Bourough contains one unassigned Neighbourhood that needs to be amended before proceeding  (Index 120)

#### Assigning Borough "cell" value to Neighbourhood not assigned "cell" value

In [50]:
tt['Neighbourhood'][(tt['Neighbourhood'] == 'Not assigned')] = tt['Borough'][(tt['Neighbourhood'] == 'Not assigned')]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


#### Last check

In [51]:
print(tt[(tt['Borough'] == 'Not assigned')].index.values)
print(tt[(tt['Neighbourhood'] == 'Not assigned')].index.values)

[]
[]


In [52]:
tt.rename(columns={'Postcode':'PostalCode'}, inplace=True)
tt.reset_index(drop=True, inplace=True)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


There is no presence of not assigned neighbourhoods or boroughs 

In [53]:
tt.head(11)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [54]:
tt.shape

(103, 3)

# Second assignment -------------------------------------------------------------------

In [55]:
!conda install -c conda-forge geocoder --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0 anaconda --> 2019.9.11-py36_0  conda-forge

The following packages will be DOWNGRADED:

    openssl: 1.1.1-h7b6447c_0 anaconda --> 1.1.1c-h516909a_0 conda-forge


certifi-2019.9.11    | 147 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


#### Using the Geospatial data file from cocl

In [56]:
!wget -q -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data

In [57]:
geo = pd.read_csv('Geospatial_data.csv')

In [58]:
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [59]:
tt.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [60]:
tt['Latitude'] = tt.PostalCode.map(geo.set_index('Postal Code')['Latitude'].to_dict())
tt['Longitude'] = tt.PostalCode.map(geo.set_index('Postal Code')['Longitude'].to_dict())

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [61]:
tt.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [62]:
tt.shape

(103, 5)

# Third assignment

In [63]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [64]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



#### Install libraries

In [65]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium # map rendering library


#### Get Toronto latitude and longitude

In [67]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create Map

In [68]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(tt['Latitude'], tt['Longitude'], tt['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare credentials

In [69]:
CLIENT_ID = 'HJLPKNFYSMPOPHISTM0LKEJHZC1MK2ENDRP1RWT5NAWSYMCN' # your Foursquare ID
CLIENT_SECRET = 'G1I1EM0WQWTCR0JH0H334RF5FJMZMTJGXSLICV01B2P2MMMU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Definition of function to loop through the different neighbourhoods extracting data from foursquare

In [70]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Getting data from nearby venues

In [71]:
tv = getNearbyVenues(names=tt['Neighbourhood'], latitudes=tt['Latitude'], longitudes=tt['Longitude'])
backup = tv


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

#### One hot encoding

In [73]:
th = pd.get_dummies(tv[['Venue Category']], prefix="", prefix_sep="")
th.shape

(2255, 270)

#### Add neighborhood column back to dataframe

In [74]:
th['Neighbourhood'] = tv['Neighborhood'] 
th.shape

(2255, 271)

#### Move neighborhood column to the first column

In [75]:
# move neighborhood column to the first column
fixed_columns = [th.columns[-1]] + list(th.columns[:-1])
th = th[fixed_columns]

#### Grouping by neighbourhood by taking the mean of the frequency of occurrence

In [76]:
tg = th.groupby('Neighbourhood').mean().reset_index()
tg.shape

(99, 271)

#### Top venues per neighbourhood

In [77]:
num_top_venues = 5

for hood in tg['Neighbourhood']:
    print("----"+hood+"----")
    temp = tg[tg['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3  Thai Restaurant  0.04
4           Bakery  0.03


----Agincourt----
               venue  freq
0             Lounge  0.25
1     Breakfast Spot  0.25
2       Skating Rink  0.25
3     Clothing Store  0.25
4  Accessories Store  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
               venue  freq
0         Playground  0.33
1             Bakery  0.33
2               Park  0.33
3  Accessories Store  0.00
4  Mobile Phone Shop  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Pizza Place  0.22
1         Grocery Store  0.22
2              Pharmacy  0.11
3  Fast Food Restaurant  0.11
4        Sandwich Place  0.11


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1  Sandwich Place  0.12
2

#### Define function to get most common venues

In [78]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Apply function

In [79]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nvs = pd.DataFrame(columns=columns)
nvs['Neighbourhood'] = tg['Neighbourhood']

for ind in np.arange(tg.shape[0]):
    nvs.iloc[ind, 1:] = return_most_common_venues(tg.iloc[ind, :], num_top_venues)

nvs.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,Bakery,Sushi Restaurant,Hotel,Asian Restaurant,American Restaurant
1,Agincourt,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Bakery,Park,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Sandwich Place,Pharmacy,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Coffee Shop,Skating Rink,Pub,Gym,Sandwich Place,Dumpling Restaurant,Drugstore,Donut Shop


#### Run K-Means

In [80]:
# set number of clusters
kclusters = 5

tgc = tg.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tgc)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Add clustering labels

In [81]:
nvs.insert(0, 'Cluster Labels', kmeans.labels_)

tm = tt
tm = tm.join(nvs.set_index('Neighbourhood'), on='Neighbourhood')
tm = tm.drop('PostalCode', 1)

In [82]:
tm.head()


,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4.0,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Pizza Place,Medical Center,Rental Car Location,Breakfast Spot,Mexican Restaurant,Electronics Store,Intersection,Eastern European Restaurant,Dumpling Restaurant,Drugstore
3,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Fried Chicken Joint,Bakery,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bank,Caribbean Restaurant,Doner Restaurant,Discount Store,Dog Run


In [83]:
tm.shape

(103, 15)

#### Check for null values and transform doubles into integers for Cluster labels

In [85]:
tm['Cluster Labels'].isnull().values

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False])

There are some values missing in the Cluster Labels, we need to drop them

In [86]:
tm = tm.dropna()

In [87]:
tm.shape

(99, 15)

In [88]:
tm['Cluster Labels'] =tm['Cluster Labels'].astype(int)

#### Map creation

In [89]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tm['Latitude'], tm['Longitude'], tm['Neighbourhood'], tm['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters